In [1]:
import time, re, datetime, os, glob
from datetime import timedelta
import seaborn as sns
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import pandas as pd
from IPython import embed as shell

## INITIALIZE A FEW THINGS
sns.set_style("darkgrid", {'xtick.bottom': True, 'ytick.left': True, 'lines.markeredgewidth':0 } )
sns.set_context(context="paper")

## CONNECT TO DJ
import datajoint as dj
from ibl_pipeline import reference, subject, action, acquisition, data, behavior
from ibl_pipeline.analyses.load_mouse_data_datajoint import get_water, get_weights, get_water_weight, get_behavior

Connecting anneurai@datajoint.internationalbrainlab.org:3306


In [2]:
# get all the mice from CSHL (exclude those with undefined sex, i.e. example animals)
subjects = pd.DataFrame.from_dict((subject.Subject() & 'responsible_user = "valeria"' & 'sex != "U"').fetch(
    order_by='subject_nickname', as_dict=True))
print(subjects['subject_nickname'].unique())

['IBL_1' 'IBL_10' 'IBL_11' 'IBL_12' 'IBL_13' 'IBL_14' 'IBL_15' 'IBL_16'
 'IBL_17' 'IBL_18' 'IBL_19' 'IBL_2' 'IBL_20' 'IBL_21' 'IBL_22' 'IBL_23'
 'IBL_24' 'IBL_25' 'IBL_26' 'IBL_27' 'IBL_28' 'IBL_29' 'IBL_3' 'IBL_30'
 'IBL_31' 'IBL_32' 'IBL_33' 'IBL_34' 'IBL_35' 'IBL_36' 'IBL_37' 'IBL_38'
 'IBL_39' 'IBL_4' 'IBL_40' 'IBL_41' 'IBL_42' 'IBL_43' 'IBL_44' 'IBL_45'
 'IBL_5' 'IBL_6' 'IBL_7' 'IBL_8' 'IBL_9']


In [6]:
#for mousename in subjects['subject_nickname']:
    
mousename = 'IBL_47'
weiwa, bl   = get_water_weight(mousename)
behav       = get_behavior(mousename)
trialcounts = behav.groupby(['date'])['choice'].count().reset_index()

# set the same format for both date axes
weiwa['date'] = weiwa['date'].astype('datetime64[ns]')
trialcounts['date'] = trialcounts['date'].astype('datetime64[ns]')

# MERGE THE TABLES TOGETHER BY DATE
data = weiwa.merge(trialcounts, on='date', how='outer')

# RENAME AND CLEAN UP
data['trialcount'] = data.choice
data.drop(columns=['choice', 'days'], inplace=True)
df = data.groupby('date').agg({'weight':'mean', 'water_administered':'sum',
                            'trialcount':'mean', 'water_type':'unique'}).reset_index()
df.sort_values('date', inplace=True)

df.tail(n=10)

,date,weight,water_administered,trialcount,water_type
24,2018-11-06,23.13,0.0000,584.0,[nan]
25,2018-11-07,22.69,0.0000,976.0,[nan]
26,2018-11-08,24.12,0.2500,342.0,[10% sucrose]
27,2018-11-09,23.75,0.0000,443.0,[nan]
28,2018-11-12,NaN,0.0000,970.0,[nan]
29,2018-11-13,23.79,1.7934,1137.0,[10% sucrose]
30,2018-11-14,23.56,1.5840,1033.0,[Water]
31,2018-11-15,23.90,1.5029,1092.0,[Water]
32,2018-11-16,23.56,1.4166,1040.0,[Water]
33,2018-11-20,22.38,1.3277,NaN,[Water]


In [7]:
# COMPUTE, FOR EACH MONDAY AND FRIDAY, THE CORRESPONDING WEEKEND WATER REGIME
df.index = df['date']
df2 = df.reindex(pd.date_range(df['date'].min(), df['date'].max()))
df2['date'] = df2.index
df2.reset_index(inplace=True)
df2['dayofweek'] = df2['date'].dt.day_name()
df2.drop(columns='index', inplace=True)

df2.tail(n=20)

,date,weight,water_administered,trialcount,water_type,dayofweek
36,2018-11-01,22.580,0.0000,586.0,[nan],Thursday
37,2018-11-02,23.325,0.0000,204.0,[nan],Friday
38,2018-11-03,NaN,NaN,NaN,NaN,Saturday
39,2018-11-04,NaN,NaN,NaN,NaN,Sunday
40,2018-11-05,24.100,0.0000,262.0,[nan],Monday
41,2018-11-06,23.130,0.0000,584.0,[nan],Tuesday
42,2018-11-07,22.690,0.0000,976.0,[nan],Wednesday
43,2018-11-08,24.120,0.2500,342.0,[10% sucrose],Thursday
44,2018-11-09,23.750,0.0000,443.0,[nan],Friday
45,2018-11-10,NaN,NaN,NaN,NaN,Saturday


In [8]:
df2['previousWater'] = df2['water_type'].shift(1)
df2['nextWater'] = df2['water_type'].shift(-1)

df2['weekend_water'] = None
df2.loc[df2.dayofweek == 'Friday', 'weekend_water'] = df2.loc[df2.dayofweek == 'Friday', 'nextWater']
df2.loc[df2.dayofweek == 'Monday', 'weekend_water'] = df2.loc[df2.dayofweek == 'Monday', 'previousWater']
# df2 = df2.loc[df2['dayofweek'].isin(['Monday', 'Friday'])]

df2.tail(n=10)

,date,weight,water_administered,trialcount,water_type,dayofweek,previousWater,nextWater,weekend_water
46,2018-11-11,NaN,NaN,NaN,NaN,Sunday,NaN,[nan],None
47,2018-11-12,NaN,0.0000,970.0,[nan],Monday,NaN,[10% sucrose],NaN
48,2018-11-13,23.79,1.7934,1137.0,[10% sucrose],Tuesday,[nan],[Water],None
49,2018-11-14,23.56,1.5840,1033.0,[Water],Wednesday,[10% sucrose],[Water],None
50,2018-11-15,23.90,1.5029,1092.0,[Water],Thursday,[Water],[Water],None
51,2018-11-16,23.56,1.4166,1040.0,[Water],Friday,[Water],NaN,NaN
52,2018-11-17,NaN,NaN,NaN,NaN,Saturday,[Water],NaN,None
53,2018-11-18,NaN,NaN,NaN,NaN,Sunday,NaN,NaN,None
54,2018-11-19,NaN,NaN,NaN,NaN,Monday,NaN,[Water],NaN
55,2018-11-20,22.38,1.3277,NaN,[Water],Tuesday,NaN,NaN,None


In [22]:
## PLOT WEIGHT
Friday = df2.loc[df2['dayofweek'] == 'Friday', :].reset_index()
Monday = df2.loc[df2['dayofweek'] == 'Monday', :].reset_index()

# useLoc = ((Friday['water_type'] == Monday['water_type']))
#Friday = Friday.loc[useLoc, :]
# Monday = Monday.loc[useLoc, :]

Friday.head(n=10)
Monday.head(n=10)

summarydat = Friday[['weekend_water', 'water_type']]
summarydat['weight'] = Monday['weight'] - Friday['weight']
summarydat['water_administered'] = Monday['water_administered'] - Friday['water_administered']
summarydat['trialcount'] = Monday['trialcount'] - Friday['trialcount']

summarydat.head(n=10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.ht

,weekend_water,water_type,weight,water_administered,trialcount
0,NaN,NaN,NaN,NaN,NaN
1,NaN,[Water],NaN,NaN,NaN
2,NaN,[Water],1.786667,-0.469,NaN
3,NaN,[Water],1.615000,0.906,6.0
4,NaN,[Water],5.145000,-0.252,-39.0
5,NaN,[nan],0.775000,0.000,58.0
6,NaN,[nan],NaN,0.000,527.0
7,NaN,[Water],NaN,NaN,NaN
